#### Fine Tuning over GoogLeNet for Stanford Mobile Image dataset. 

In [1]:
aws = False  #if it is working over AWS 

import matplotlib
if aws:
    matplotlib.use('Pdf')
    user = 'ubuntu'
else:
    user = 'payam'
import numpy as np
from pylab import *
%matplotlib inline
import tempfile

caffe_root = '/home/{0}/workspace/caffe/'.format(user)
dataset_root = '/home/{0}/dataset/stanford_mobile_images/'.format(user)

import sys
sys.path.insert(0, caffe_root + 'python')
import caffe
if aws:
    caffe.set_device(0) #if run on g2.8xlarge turn 0  to all
    caffe.set_mode_gpu()
else:
    caffe.set_mode_cpu()
    
#network
model_name = 'bvlc_googlenet'
model_prototxt = "{0}models/{1}/my_train_val.prototxt".format(caffe_root, model_name)
model_trained = "{0}models/{1}/{1}.caffemodel".format(caffe_root, model_name)

#### my input layer

In [2]:
import os
def file_formatter(folder):
    """
    This function prepare files for caffe input.
    It gets folder name, and creates a text file with list of images in the given folder.
    And returns text file name.
    """
    counter = 0
    with open(folder + '/images.txt', 'w') as output:
        for f in os.listdir(folder):
            if os.path.isfile(folder + '/' + f) and f.endswith('.jpg'):
                output.write('{0}/{1} {1}\n'.format(folder, f))
                #in the format of: /path/to/file/0xx.jpg 0xx.jpg(class)
                counter += 1
    return '{0}/{1}'.format(folder, 'images.txt'), counter

In [3]:
def disp_preds(net, image, labels, k=5, name='ImageNet'):
    input_blob = net.blobs['data']
    net.blobs['data'].data[0, ...] = image
    probs = net.forward(start='conv1')['probs'][0]
    top_k = (-probs).argsort()[:k]
    print 'top %d predicted %s labels =' % (k, name)
    print '\n'.join('\t(%d) %5.2f%% %s' % (i+1, 100*probs[p], labels[p])
                    for i, p in enumerate(top_k))

def disp_imagenet_preds(net, image):
    disp_preds(net, image, imagenet_labels, name='ImageNet')

def disp_style_preds(net, image):
    disp_preds(net, image, style_labels, name='style')

In [4]:
net = caffe.Classifier(model_prototxt, model_trained,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

IndexError: list index out of range